In [ ]:
# Importing Libraries
import cv2
import mediapipe as mp
from math import hypot
import screen_brightness_control as sbc
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Initialize Mediapipe Hands
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2,
)
Draw = mp.solutions.drawing_utils

# Initialize Pycaw for Volume Control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None
)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    # Read video frame by frame
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip image
    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB

    # Process the RGB image
    process = hands.process(frameRGB)

    landmarkList = []
    if process.multi_hand_landmarks:
        for hand_id, handlm in enumerate(process.multi_hand_landmarks):
            hand_type = "Left" if hand_id == 0 else "Right"  # Assuming left hand is first
            for _id, landmarks in enumerate(handlm.landmark):
                height, width, color_channels = frame.shape
                x, y = int(landmarks.x * width), int(landmarks.y * height)
                landmarkList.append([hand_type, _id, x, y])

            # Draw landmarks
            Draw.draw_landmarks(frame, handlm, mpHands.HAND_CONNECTIONS)

    # Handle brightness and volume control
    if landmarkList != []:
        for hand in ["Left", "Right"]:
            hand_points = [lm for lm in landmarkList if lm[0] == hand]
            if hand_points:
                x_thumb, y_thumb = hand_points[4][2], hand_points[4][3]  # Thumb tip
                x_index, y_index = hand_points[8][2], hand_points[8][3]  # Index finger tip

                # Draw circles and line
                cv2.circle(frame, (x_thumb, y_thumb), 7, (0, 255, 0), cv2.FILLED)
                cv2.circle(frame, (x_index, y_index), 7, (0, 255, 0), cv2.FILLED)
                cv2.line(frame, (x_thumb, y_thumb), (x_index, y_index), (0, 255, 0), 3)

                # Calculate distance
                length = hypot(x_index - x_thumb, y_index - y_thumb)

                if hand == "Right":
                    # Brightness control (Hand range: 15-220, Brightness range: 0-100)
                    b_level = np.interp(length, [15, 220], [0, 100])
                    sbc.set_brightness(int(b_level))
                elif hand == "Left":
                    # Volume control (Hand range: 15-220, Volume range: -65.25 to 0)
                    vol_level = np.interp(length, [15, 220], [-65.25, 0])
                    volume.SetMasterVolumeLevel(vol_level, None)

    # Display Video and Exit on 'q'
    cv2.imshow("Image", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	main.ipynb

nothing added to commit but untracked files present (use "git add" to track)
